In [1]:
import requests
from bs4 import BeautifulSoup as bs
from  tqdm.notebook import tqdm
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

import random

import time
import re

In [2]:
main_link='https://www.captainbook.gr/bookstore/year/2016?page='

headers = {'User-agent': ''}

response = requests.get(main_link, headers=headers)

if response.status_code == 403:
    print('you got caught!')


In [5]:
soup = bs(response.content, 'html.parser')

In [216]:
for_p = soup.find_all('ul', 
                      class_='pagination justify-content-end')[0].find('li', 
                                                                      class_="page_item d-flex align-items-center")

max_pages = for_p.find_all('a')[-2].get_text()

In [217]:
books = []
publisher = []
links = []

In [247]:
price_old = []
price_new = []
price_discount = []

In [255]:
for page in tqdm(range(541,int(max_pages)+1)):
    response = requests.get(main_link+str(page), headers = headers)
    if response.status_code == 200:
        soup = bs(response.content, 'html.parser')
        titles = soup.find_all('h1')
        prices = soup.find_all('div', class_='price p-0 d-flex mb-2 align-items-center justify-content-between')
        publ = soup.find_all('h2', class_='p-0')
        for i in range(12):
            try: 
                price_old.append(prices[i].find_all('span', class_="old-price p-0")[0].get_text().strip())
            except:
                price_old.append(np.nan)
            try:
                price_new.append(prices[i].find_all('span', class_="new-price p-0")[0].get_text().strip())
            except:
                price_new.append(np.nan)
            try:
                price_discount.append(prices[i].find_all('small', class_="p-0")[0].get_text().strip())
            except:
                price_discount.append(np.nan)
            try:
                publisher.append(publ[i].get_text().strip())
            except:
                publisher.append(np.nan)
            try:
                books.append(titles[i].get_text().strip())
            except:
                books.append(np.nan)
            try:
                links.append(titles[i].find('a', class_="p-0 js-title productClickTracker").get('href'))
            except:
                links.append(np.nan)
    else:
        saved = i
        print('You got caught!')
        break
    
    time.sleep(random.random())   

  0%|          | 0/446 [00:00<?, ?it/s]

In [262]:
dic = {'Title': books,
    'Price_Old': price_old,
    'Discount': price_discount,
    'Price_New': price_new,
    'Publisher': publisher,
    'Link': links }

In [263]:
df = pd.DataFrame(dic, columns=['Title','Price_Old','Discount','Price_New','Publisher','Link'])

In [271]:
df['Year'] = 2020

In [278]:
df.shape

(11832, 7)

In [275]:
df.to_csv('books_2020.csv',index=False)

In [16]:
df = pd.read_csv('../data/Books2018.csv')

In [17]:
df.head()

,Title,Price_Old,Discount,Price_New,Publisher,Link,Year
0,Οι άλλες ιστορίες,13.00€,-27%,9.49€,Marcos Subcomandante,https://www.captainbook.gr/book/oi-alles-istor...,2018
1,O ΜΑΙΓΚΡΕ ΚΑΙ Ο ΝΕΚΡΟΣ ΤΟΥ,8.80€,-57%,3.80€,Georges Simenon,https://www.captainbook.gr/book/o-maigkre-kai-...,2018
2,Το δωμάτιο του Τζοβάνι,50.00€,-11%,44.50€,James Baldwin,https://www.captainbook.gr/book/to-domatio-tou...,2018
3,Περιβαλλοντικό Ποινικό Δίκαιο,20.00€,-11%,17.80€,Γεώργιος Π. Αλμπούρας,https://www.captainbook.gr/book/periballontiko...,2018
4,Περιβαλλοντικό Ποινικό Δίκαιο,90.00€,-11%,80.10€,Γεώργιος Π. Αλμπούρας,https://www.captainbook.gr/book/periballontiko...,2018


In [23]:
category = []
ISBN = []
pages=[]
binding = []

In [32]:
for link in tqdm(df.iloc[11725:,5]):
    try:
        response = requests.get(link, headers = headers)
    
        if response.status_code == 200:
            soup1 = bs(response.content, 'html.parser')
            body = soup1.find('div', class_="fading col-md-12 p-0").find_all('div', class_="td")
            body1=soup1.find_all('li', class_="py-2 d-flex justify-content-between")
            
            try:
                count1=0
                count2=0
                count3=0
                count4=0

                for bod in body1:
                    if re.search(r'Κατηγορίες',str(bod)):
                        category.append(bod.get_text().strip().split('\n')[-1])
                        count1+=1
                    elif re.search(r'ISBN',str(bod)):
                        ISBN.append(bod.get_text().strip().split('\n')[-1])
                        count2+=1
                
                for i in range(len(body)):
                    if re.search(r'Σελίδες',str(body[i])):
                        pages.append(body[i+1].get_text().strip())
                        count3 +=1
                    elif re.search(r'Δέσιμο',str(body[i])):
                        binding.append(body[i+1].get_text().strip())
                        count4 +=1

                if count1 == 0:
                    category.append(np.nan)
                if count2 == 0:
                    ISBN.append(np.nan)
                if count3 == 0:
                    pages.append(np.nan)
                if count4 == 0:
                    binding.append(np.nan)

            except:
                saved=link
                print('dafuq')
                break

        else:
            saved=link
            print('You got caught!')
            break
    except:
        category.append(np.nan)
        ISBN.append(np.nan)
        pages.append(np.nan)
        binding.append(np.nan)
    

    time.sleep(random.uniform(0,3))

  0%|          | 0/611 [00:00<?, ?it/s]

In [33]:
len(category) , len(ISBN), len(pages), len(binding)

(12336, 12336, 12336, 12336)

In [34]:
df['Category'] = category
df['ISBN'] = ISBN
df['Pages'] = pages
df['Binding'] = binding

In [35]:
df.head()

,Title,Price_Old,Discount,Price_New,Publisher,Link,Year,Category,ISBN,Pages,Binding
0,Οι άλλες ιστορίες,13.00€,-27%,9.49€,Marcos Subcomandante,https://www.captainbook.gr/book/oi-alles-istor...,2018,", Μαρτυρίες",9789609797733,104,Μαλακό εξώφυλλο
1,O ΜΑΙΓΚΡΕ ΚΑΙ Ο ΝΕΚΡΟΣ ΤΟΥ,8.80€,-57%,3.80€,Georges Simenon,https://www.captainbook.gr/book/o-maigkre-kai-...,2018,Πεζογραφία,9789605053185,272,Χαρτόδετο
2,Το δωμάτιο του Τζοβάνι,50.00€,-11%,44.50€,James Baldwin,https://www.captainbook.gr/book/to-domatio-tou...,2018,", Νουβέλα",9786180314175,248,Χαρτόδετο
3,Περιβαλλοντικό Ποινικό Δίκαιο,20.00€,-11%,17.80€,Γεώργιος Π. Αλμπούρας,https://www.captainbook.gr/book/periballontiko...,2018,Νομική,9789605961251,738,Χαρτόδετο
4,Περιβαλλοντικό Ποινικό Δίκαιο,90.00€,-11%,80.10€,Γεώργιος Π. Αλμπούρας,https://www.captainbook.gr/book/periballontiko...,2018,Νομική,9789605961244,258,Χαρτόδετο


In [36]:
df.to_csv('../data/Books2018.csv',index=False)

In [4]:
import pickle

In [81]:
# pickle_out = open('category.pickle', 'wb')
# pickle.dump(category, pickle_out)
# pickle_out.close()

# pickle_out = open('ISBN.pickle','wb')
# pickle.dump(ISBN, pickle_out)
# pickle_out.close()

# pickle_out = open('binding.pickle','wb')
# pickle.dump(binding, pickle_out)
# pickle_out.close()

# pickle_out = open('pages.pickle','wb')
# pickle.dump(pages, pickle_out)
# pickle_out.close()

5072 last observation

In [5]:
# binding = pickle.load(open('binding.pickle','rb'))
# ISBN = pickle.load(open('ISBN.pickle','rb'))
# category = pickle.load(open('category.pickle','rb'))
# pages = pickle.load(open('pages.pickle','rb'))